In [15]:
%pip install -Uq upgini catboost


In [16]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip"
df = pd.read_csv(df_path, compression="zip")
df= df.sample(n=10_00,random_state=0)
df["store"] = df["store"].astype("str")
df["item"]= df["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()


,date,store,item,sales
0,2013-01-01,5,24,26
1,2013-01-02,2,7,47
2,2013-01-03,6,21,23
3,2013-01-05,9,37,12
4,2013-01-07,5,49,15


In [17]:
train= df[df["date"]<"2017-01-01"]
test =df[df["date"]>="2017-01-01"]

In [18]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [19]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=116e30be-e2c2-4286-aecf-381267594257
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


5 relevant feature(s) found with the search keys: ['date']


item,0.3786,100.0000,"17, 12, 43",,,
store,0.2017,100.0000,"3, 8, 5",,,
f_events_date_year_cos1_9014a856,0.1213,100.0000,"-0.809, -0.6537, 0.6273",Upgini,Calendar data,Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,0.0415,100.0000,"1.0566, 0.9476, 1.0155",Upgini,Markets data,Daily
f_events_date_week_sin1_847b5db1,0.0130,100.0000,"0.0, -0.4339, 0.0",Upgini,Calendar data,Daily


Upgini,Calendar data,0.1343,2
Upgini,Markets data,0.0415,1


In [20]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)
enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


item,13.9336,100.0000,"31, 15, 25",,,
store,6.7887,100.0000,"6, 1, 2",,,
f_events_date_year_cos1_9014a856,5.6806,100.0000,"-0.7441, 0.5582, 0.8521",Upgini,Calendar data,Daily
f_events_date_week_sin1_847b5db1,4.1067,100.0000,"-0.7818, -0.9749, 0.9749",Upgini,Calendar data,Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,2.7853,100.0000,"0.6769, 0.9882, 0.9908",Upgini,Markets data,Daily


Upgini,Calendar data,9.7873,2
Upgini,Markets data,2.7853,1


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,798,51.0501,0.320 ± 0.092,0.271 ± 0.069,0.049207
1,Eval 1,202,61.9109,0.305 ± 0.040,0.271 ± 0.079,0.034562


In [21]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 0 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-




Running search request, search_id=5df17d27-353d-4e87-860f-23b014e26b9d
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 0 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-




Running search request, search_id=80034efa-afdc-4a57-996e-517238a9fabf
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_7d_to_1y_c3e0ad17
0,2013-01-01,5,24,0.781831,0.985220,0.962707
1,2013-01-02,2,7,0.974928,0.982126,0.967773
2,2013-01-03,6,21,0.433884,0.978740,0.971298
3,2013-01-05,9,37,-0.974928,0.971100,0.978426
4,2013-01-07,5,49,0.000000,0.962309,0.985867


In [22]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[41.395789274257126]

In [23]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

[25.23091639281408]